In [ ]:
import pandas as pd

## **Presentation Outline**
 
[Link](https://docs.google.com/presentation/d/1HSTv5NgdKBrXRzOEiITktw4wbaG_nAib/edit?usp=sharing&ouid=107908789910623442905&rtpof=true&sd=true)

#### **Respondent Demographics**
* Age 
* Race 
* Geography 


#### **Respondent Educational History**
* Highest educational attainment 
    * By geography, race, age 
* Training Experience
    * How many participated in trainings
    * By topic 
    * Feedback 


#### **Respondent Employment Status**

* Current status (employed/unemployed)
    * By geography, age, educational attainment 
* Job Type 
    * By geography, age, educatinoal attainment
    

#### **Respondent Employment History**

* By geography 
* By type of job 

#### **Respondent Training Experience**

* By geography 
* By educational attainment 
* By current job 

#### **Respondent Unemployment Benefits**

* Reasons for not applying (people currently unemployed)


#### **Job Search Tools and Trends**

* trends by ideal job type

#### **Respondent Ideal Employment**

* By education 
* By training experience 
* By training desire 
* By current job type 

#### **Respondent Barriers to Employment**
* by education 
* by geography 
* by ideal job type 


#### **Skills Area Strengths**
* by education
* by ideal job 
* by training experience
* by training desires 


#### **Skills Area Gaps**
* by education
* by ideal job 
* by training experience
* by training desires 
